In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
df=pd.read_csv('/content/drive/MyDrive/revenue.csv')

In [37]:
df['Location'] = "Bangalore"

In [38]:
df['Month']=pd.to_datetime(df['Month'])

<ipython-input-38-77ca7e4e6b4e>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Month']=pd.to_datetime(df['Month'])


In [39]:
df['Revenue'] = df['Revenue'].str.replace(',', '').astype(float)

In [40]:
df['Index'] = range(1, len(df) + 1)

In [41]:
df.rename(columns={'Month':'Date'}, inplace=True)

In [42]:
df.head()

,Date,Revenue,Location,Index
0,2013-12-01,346200.0,Bangalore,1
1,2014-01-01,579522.0,Bangalore,2
2,2014-02-01,64850.0,Bangalore,3
3,2014-03-01,1907743.0,Bangalore,4
4,2014-04-01,45500.0,Bangalore,5


In [43]:
df.tail()

,Date,Revenue,Location,Index
122,2024-02-01,8058888.0,Bangalore,123
123,2024-03-01,8058249.0,Bangalore,124
124,2024-04-01,8823294.0,Bangalore,125
125,2024-05-01,7636227.0,Bangalore,126
126,2024-06-01,8543991.0,Bangalore,127


In [44]:
categorical_covariates = ['Index']

categorical_covariates_num_embeddings = []
for col in categorical_covariates:
  df[col] = df[col].astype('category').cat.codes
  categorical_covariates_num_embeddings.append(df[col].nunique())

categorical_static = ['Location']

categorical_static_num_embeddings = []
for col in categorical_static:
  df[col] = df[col].astype('category').cat.codes
  categorical_static_num_embeddings.append(df[col].nunique())

numeric_covariates = ['Revenue']

target_idx = np.where(np.array(numeric_covariates) == 'Revenue')[0][0]

In [45]:
target_idx

0

In [46]:
df.head()

,Date,Revenue,Location,Index
0,2013-12-01,346200.0,0,0
1,2014-01-01,579522.0,0,1
2,2014-02-01,64850.0,0,2
3,2014-03-01,1907743.0,0,3
4,2014-04-01,45500.0,0,4


In [47]:
df.tail()

,Date,Revenue,Location,Index
122,2024-02-01,8058888.0,0,122
123,2024-03-01,8058249.0,0,123
124,2024-04-01,8823294.0,0,124
125,2024-05-01,7636227.0,0,125
126,2024-06-01,8543991.0,0,126


In [58]:
def dataframe_to_tensor(df, numeric_covariates, categorical_covariates, categorical_static, target_idx):
  numeric_cov_arr = np.array(df[numeric_covariates].values.tolist())
  category_cov_arr = np.array(df[categorical_covariates].values.tolist())
  static_cov_arr = np.array(df[categorical_static].values.tolist())

  x_numeric = torch.tensor(numeric_cov_arr, dtype=torch.float32).transpose(2, 1)
  x_numeric = torch.log(x_numeric + 1e-5)
  x_category = torch.tensor(category_cov_arr, dtype=torch.long).transpose(2, 1)
  x_static = torch.tensor(static_cov_arr, dtype=torch.long)
  y = torch.tensor(numeric_cov_arr[:, target_idx], dtype=torch.float32)

  return x_numeric, x_category, x_static, y


window_size = 16
forecast_length = 16
num_val = 2

val_max_date = '2024-06-01'
train_max_date = str((pd.to_datetime(val_max_date) - pd.Timedelta(days=(window_size*num_val+forecast_length)*30)).date())

train_final = df[df['Date']<=train_max_date]
val_final = df[(df['Date']>train_max_date) & (df['Date']<=val_max_date)]

train_series = train_final.groupby(['Location']).agg(list).reset_index()
val_series = val_final.groupby(['Location']).agg(list).reset_index()

x_numeric_train_tensor, x_category_train_tensor, x_static_train_tensor, y_train_tensor = dataframe_to_tensor(train_series, numeric_covariates, categorical_covariates, categorical_static, target_idx)
x_numeric_val_tensor, x_category_val_tensor, x_static_val_tensor, y_val_tensor = dataframe_to_tensor(val_series, numeric_covariates, categorical_covariates, categorical_static, target_idx)


In [61]:
df.shape[0]

127

In [62]:
def divide_shuffle(df, div_num):
  space = df.shape[0]//div_num
  division = np.arange(0, df.shape[0], space)
  return pd.concat([df.iloc[division[i]:division[i]+space, :].sample(frac=1) for i in range(len(division))])

def create_time_blocks(time_length, window_size, forecast_length):
  start_idx = np.randint(0, window_size - 1)
  end_idx = time_length - window_size - forecast_length - 1
  time_indices = np.arange(start_idx, end_idx+1, window_size)[:-1]
  time_indices = np.append(time_indices, end_idx)
  return time_indices
